Реализовать с использованием потоков и процессов скачивание файлов из интернета. 
Список файлов для скачивания подготовить самостоятельно (например изображений, не менее 100 изображений или других объектов). Сравнить производительность с последовательным методом. Сравнивть производительность Thread и multiprocessing решений. Попробовать подобрать оптимальное число потоков/процессов. 

Создадим список ссылок на картинки, которые будут скачиваться.

In [86]:
import requests
from bs4 import BeautifulSoup

url = "https://unsplash.com/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
image_tags = soup.find_all('img')
image_links = [img['src'] for img in image_tags if 'src' in img.attrs] # Выбираем ссылки на изображения
absolute_links = [requests.compat.urljoin(url, link) for link in image_links] # Делаем ссылки абсолютными




Реализуем метод download, выполняющий поочередное скачивание изображений и помещающий их в папку.

In [91]:
import urllib.request
import os
import time

#dirname = "download"
#file_path = os.makedirs(dirname, exist_ok=True)
def delete_files_in_folder(folder_path): # Метод, удаляющий файлы из каталога после загрузки
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f'Ошибка при удалении файла {file_path}. {e}')

def download (links, thread_number):
    dirname = "download"
    object_counter = 0
    file_path = os.makedirs(dirname, exist_ok=True) # Создаем папку для файлов
    for i, link in enumerate(links): # Загружаем файлы
        try:
                response = requests.get(link)
                file_name = f"image_{i}_{thread_number}.jpg"
                if not file_name.endswith('.jpg'): 
                    file_name += '.jpg'
                save_path = os.path.join(dirname, file_name)

                with open(save_path, 'wb') as file:
                    file.write(response.content)
                object_counter += 1
        except requests.RequestException as e:
            continue
    print(f"Поток {thread_number} загрузил {object_counter} объектов")




In [92]:
# Запускаем скачивание с 1 потоком и одним процессом, замеряем время выполнения
start_time = time.time()
download (absolute_links,0)
end_time = time.time()

print(f"Время выполнения: {end_time - start_time} секунд")

Поток 0 загрузил 114 объектов
Время выполнения: 30.479105472564697 секунд


In [93]:
import threading
def run_with_threads(num_threads):
    threads = []
    
    last_item = 0
    batch_size = (len(absolute_links)//num_threads)
    for i in range(num_threads):
        # Создаем поток для выполнения задачи
        
        thread = threading.Thread(target=download, args=(absolute_links[last_item:last_item+batch_size],i,))
        threads.append(thread)
       # thread.start()  # Запускаем поток
        last_item += batch_size
        
    for thread in threads:
        thread.start()

    
    # Ждем завершения всех потоков
    for thread in threads:
        thread.join()

# Цикл, который будет запускать код с разным количеством потоков
times_of_execution_threads = {}
for num_threads in [2,4,8,10,12]:  # Запускаем загрузку с различным числом потоков
    print(f"\nЗапуск с {num_threads} потоками...")
    start_time = time.time()  # Засекаем время начала
    run_with_threads(num_threads)
    delete_files_in_folder("download") 
    end_time = time.time()  # Засекаем время окончания
    print(f"Время выполнения с {num_threads} потоками: {end_time - start_time} секунд")
    times_of_execution_threads[num_threads] = end_time - start_time
        
#print (times_of_execution_threads)


Запуск с 2 потоками...
Поток 1 загрузил 52 объектов
Поток 0 загрузил 62 объектов
Время выполнения с 2 потоками: 18.63957118988037 секунд

Запуск с 4 потоками...
Поток 1 загрузил 33 объектов
Поток 3 загрузил 27 объектов
Поток 0 загрузил 29 объектов
Поток 2 загрузил 25 объектов
Время выполнения с 4 потоками: 9.626161336898804 секунд

Запуск с 8 потоками...
Поток 2 загрузил 16 объектов
Поток 1 загрузил 17 объектов
Поток 3 загрузил 17 объектов
Поток 6 загрузил 12 объектов
Поток 7 загрузил 15 объектов
Поток 5 загрузил 13 объектов
Поток 4 загрузил 12 объектов
Поток 0 загрузил 12 объектов
Время выполнения с 8 потоками: 5.004340410232544 секунд

Запуск с 10 потоками...
Поток 6 загрузил 10 объектов
Поток 3 загрузил 14 объектов
Поток 5 загрузил 10 объектов
Поток 1 загрузил 12 объектов
Поток 2 загрузил 13 объектов
Поток 0 загрузил 10 объектов
Поток 7 загрузил 10 объектов
Поток 8 загрузил 10 объектов
Поток 4 загрузил 13 объектов
Поток 9 загрузил 12 объектов
Время выполнения с 10 потоками: 5.45496

In [94]:
# Выведем результаты работы разного количества потоков
for key in times_of_execution_threads.keys():
    print(f"Время выполнения с {key} потоками - {times_of_execution_threads[key]} секунд")

Время выполнения с 2 потоками - 18.63957118988037 секунд
Время выполнения с 4 потоками - 9.626161336898804 секунд
Время выполнения с 8 потоками - 5.004340410232544 секунд
Время выполнения с 10 потоками - 5.454962253570557 секунд
Время выполнения с 12 потоками - 5.862089395523071 секунд


Наиболее быстро скачивание происходит при работе с 8 потоками, при этом ускорение скачивания нелинейно и различается между запусками скрипта. Оптимальное число потоков - 8-10.

Для оценки скорости загрузки с разным числом запущенных процессов был запущен код в файле prosesses.py.
Результат работы различного числа процессов:

Время выполнения с 2 процессами - 18.145548105239868 секунд
Время выполнения с 4 процессами - 11.212659358978271 секунд
Время выполнения с 8 процессами - 6.246687412261963 секунд
Время выполнения с 10 процессами - 6.31508994102478 секунд

Наиболее быстро скачивание происходит при работе с 8 процессами, однако увеличение числа потоков дает больший прирост производительности, чем увеличение числа процессов.
В целом, увеличение количества процессов/потоков ускоряет процесс скачивания, за счет избегания ожидания ответа сервера на каждый запрос по очереди, а параллельного выполнения запросов.